In [3]:
import numpy as np

def first_x_sec(seconds, filepath, timecol):
    ''' param: seconds, int, filter out the first seconds of the game
        param: filepath, string, the filepath of the csv datafile
        param: timecol, int, the index of the column that contains time
    '''
    f = open(filepath)
    lines = f.readlines()
    lines = [ line.split(',') for line in lines ]
    if seconds is None:
        return np.array(lines[1:])
    else:
        return np.array(list(filter(lambda line: int(line[timecol])<=seconds, lines[1:])))

# First 5 min: 5*60=300, col 2 is time first line is header
#ability_upgrade = first_x_sec(300, 'dataset/ability_upgrades.csv', 2)
#objectives = first_x_sec(300, 'dataset/objectives.csv', 7)
player_time = first_x_sec(300, 'dataset/player_time.csv', 1)


## Match file contains more than just the label
# column 10 contains the True/False value of Radiant win
labels = first_x_sec(None, 'dataset/match.csv', None)
purchases = first_x_sec(300, 'dataset/purchase_log.csv', 1)
print(purchases)

[['44' '-81' '0' '0\n']
 ['29' '-63' '0' '0\n']
 ['43' '6' '0' '0\n']
 ..., 
 ['77' '150' '132' '49999\n']
 ['20' '150' '132' '49999\n']
 ['25' '284' '132' '49999\n']]


In [4]:
labels = np.array([int(match[9] == 'True') for match in labels])
print('All the labels')
print(labels)

All the labels
[1 0 0 ..., 1 1 0]


In [5]:
import math
print(purchases.shape)
print(purchases[1])
purchases = purchases.astype(np.int)

#def set_forth(log):
#        log[1] = math.ceil(max(0, log[1]) / 60) * 60
#        return log
    
for log in purchases:
    log[1] = math.ceil(max(0, log[1]) / 60) * 60

#purchases = np.array([ set_forth(log) for log in purchases ])    
print(purchases)



(4014805, 4)
['29' '-63' '0' '0\n']
[[   44     0     0     0]
 [   29     0     0     0]
 [   43     0     0     0]
 ..., 
 [   77   120   132 49999]
 [   20   120   132 49999]
 [   25   240   132 49999]]


In [9]:
#in any game, any player for one timestep, what is maximum number of items?
matches = [np.empty(shape=(4,0), dtype=np.int)]*50000
for x in purchases:
    matches[x[3]] = np.concatenate((matches[x[3]], x.reshape(4,1)), axis=1)

# separate on time
#print(matches[1].T)
for i, match in enumerate(matches):
    separated = [np.empty(shape=(4,0), dtype=np.int)]*6 # 6 time steps [0 60 120 180 240 300]
    for x in match.T:
        time = x[1]/60
        separated[time] = np.concatenate((separated[time], x.reshape(4,1)), axis=1)
    matches[i] = separated

#Separate on player
for match in matches:
    for i, time_step in enumerate(match):
        #problem is that this vector is the same for all players. Need 10 copies
        separated = [np.zeros(shape=(218,1), dtype=np.int) for j in range(10)]*10 # 10 players and 218 is maximum item id in dataset
        
        for x in time_step.T:
            player = x[2] if x[2]<5 else x[2] - 123 #make players numbered 0 to 9
            separated[player][x[0]-1] += 1
        match[i] = separated
print(423)      

423


In [10]:
print(4)

4


In [20]:
print matches[15][0][2].shape # first match, first timestep 0 and first player purchase log

(218, 1)


[0]
maximum unique items bought an one timestep
1
1


In [35]:
print('Shape of player time matrix')
print('Contains all attributes of matches, the first minutes')
print(player_time.shape)
#print(player_time[1:10])
player_time = player_time.astype(np.int)


## Separate data matches. Each index in data is one match. One match is a 32*n.
## n is the number of logs. We have data every 60 secods, so for 5 min n=5
data = [np.empty(shape=(2212,0), dtype=np.int)]*50000
for x in player_time:
    timestep = x[1]/60
    features = x.reshape(32, 1)
    for player in range(10):
        features = np.concatenate((features, matches[x[0]][timestep][player]), axis=0)   
    data[x[0]] = np.concatenate((data[x[0]], features), axis=1)
#Print the first match log 
print('The pre prossessed data, just the second match')
print(data[1])


 Shape of player time matrix
Contains all attributes of matches, the first minutes
(299994, 32)
The pre prossessed data, just the second match
[[  1   1   1   1   1   1]
 [  0  60 120 180 240 300]
 [  0  99 199 299 399 798]
 ..., 
 [  1   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]]


In [65]:
#Pad games shorter than timelimit. (5 minutes)
for i in range(len(data)):
    while data[i].shape[1] <= 300/60:
        features = np.zeros(shape=(2212,1), dtype=np.int)
        data[i] = np.concatenate((data[i], features), axis=1)
    

In [78]:
outfile = open('/tmp/game_data.npy', 'w')
np.save(outfile, data)
outfile.close()  

In [79]:
outfile = open('/tmp/game_labels.npy', 'w')
labels = labels.reshape((50000, 1))
np.save(outfile, labels) 
outfile.close()

Shapes of labels
(50000,)
example of how the labels look like
[1 0 0 0 1 1 1 1 0 0]
Re-structure the labels so that it is two dimensions, with prob value of each team winning
(50000, 2)
example of how the binary labels look like
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
(10000, 2)


Train on 7999 samples, validate on 32001 samples
Epoch 1/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 2/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 3/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 4/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 5/50
7999/7999 [==============================] - 0s 

7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 38/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 39/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 40/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categorical_accuracy: 0.5188 - val_loss: 7.7948 - val_mean_absolute_error: 0.4836 - val_categorical_accuracy: 0.5164
Epoch 41/50
7999/7999 [==============================] - 0s - loss: 7.7558 - mean_absolute_error: 0.4812 - categoric

 7488/10000 [=====================>........] - ETA: 0s[7.6399250358581545, 0.47409038228988648, 0.52590000000000003]
